In [ ]:
from gurobipy import *
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
n=8 #No. customers
f=3 #No. fuel stations
nc=[i for i in range(1,n+1)]
nf=[i for i in range(n+1,n+f+1)]
v=[0]+nc+nf
print(v)

xc=[40, 30,58,48,40,25,  39,15,40] #X-cor of nodes
yc=[50, 50,75,40,15,40,  26,70,60] #Y-cor of nodes
r=[0,20,10,30,20,20,0,0,0] #Demand of nodes

cap=80 #Vehicle load capacity
ro=1 #consumption rate
F=120 #Battrey capacity
m=np.ceil(sum(r)/cap) #No of Vehicles



#Plot nodes
plt.scatter(xc[1:n+1],yc[1:n+1],c='b')
plt.scatter(xc[n+1:],yc[n+1:],c='g')
plt.scatter(xc[0],yc[0],c='r')

In [ ]:
A=[(i,j) for i in v for j in v ] #Arcs

c={(i,j):np.hypot(xc[i]-xc[j],yc[i]-yc[j]) for i,j in A} #Distance matrix



In [ ]:

model=Model('EVRP')

x = model.addVars(A,vtype=GRB.BINARY,name='x') #Equal 1 if vehicle goes from node i to node j
w = model.addVars(A,vtype=GRB.CONTINUOUS,name='w') #Load on Arc i>j
q = model.addVars(v,vtype=GRB.CONTINUOUS,name='q') #State  of charge when vehicle arrives at fuel station
y = model.addVars(v,vtype=GRB.CONTINUOUS,name='y') #State  of charge when vehicle leave node

In [ ]:
# model.setObjective(quicksum(c[i,j]*x[i,j] for i,j in A  if i!=j) , GRB.MINIMIZE)

model.setObjective(quicksum(c[i,j]*x[i,j] for i,j in A  if i!=j) + quicksum(1000*x[0,i] for i in v) , GRB.MINIMIZE)

model.addConstrs(quicksum(x[i,j] for j in v if j!=i)==1 for i in nc)

model.addConstrs(quicksum(x[i,j] for j in v if j!=i)<=1 for i in nf )

model.addConstrs(quicksum(x[j,i] for i in v if i!=j) -quicksum(x[i,j] for i in v if i!=j)==0 for j in v)

model.addConstrs(quicksum(w[j,i] for j in v if i!=j) - quicksum(w[i,j] for j in v if i!=j) == r[i] for i in nc+nf)

model.addConstrs(w[i,j]<=cap*x[i,j] for i,j in A if i!=j)

model.addConstrs(ro*c[i,j]*x[i,j]-F*(1-x[i,j])<=y[i]-y[j] for i in v for j in nc)

model.addConstrs(y[i]-y[j]<=ro*c[i,j]*x[i,j]+F*(1-x[i,j]) for i in v for j in nc)

model.addConstrs(ro*c[i,j]*x[i,j]-F*(1-x[i,j])<=y[i]-q[j] for i in v for j in nf)

model.addConstrs(y[i]-q[j]<=ro*c[i,j]*x[i,j]+F*(1-x[i,j]) for i in v for j in nf)

model.addConstrs(y[i]>=ro*c[i,0]*x[i,0] for i in nc+nf)

model.addConstrs(y[i]==F for i in [0]+nf)

model.Params.TimeLimit = 300
model.Params.MIPGap = 0
model.Params.Threads = 2
model.optimize()

In [ ]:
print("Status: ", str(model.Status))
print("Objective Value: ", str(np.round(model.ObjVal,2)))
print("CPU Time: ", str(np.round(model.Runtime,2)))

In [ ]:
K = 0

for i in v:
    if i!=0 and x[0,i].x>0.9:
        K+=1
K

In [ ]:
routes=[]


for i in v:
    if i!=0 and x[0,i].x>0.9:
        aux=[0,i]
        while i!=0:
            j=i
            for h in v:
                if j!=h and x[j,h].x>0.9:
                    aux.append(h)
                    i=h
        routes.append(aux)

print(routes)


In [ ]:
plt.scatter(xc[1:n+1],yc[1:n+1],c='b')
plt.scatter(xc[n+1:],yc[n+1:],c='g')
plt.scatter(xc[0],yc[0],c='r')
for i in nc:
    plt.text(xc[i],yc[i]+3,"C"+format(i))
    
for i in nf:
    plt.text(xc[i],yc[i]+3,"F"+format(i))


for k in range(0,len(routes)):
    for i in range(1,len(routes[k])):
        plt.annotate(s='',xy=(xc[routes[k][i]],yc[routes[k][i]]),xytext=(xc[routes[k][i-1]],yc[routes[k][i-1]]), arrowprops=dict(arrowstyle='->'))
        # plt.annotate(s='',xy=(xc[j],yc[j]),xytext=(xc[i],yc[i]), arrowprops=dict(arrowstyle='->'))